In [ ]:
# Make workspace
%env AZURE_EXTENSION_DIR=/home/schrodinger/automl/sdk-cli-v2/src/cli/src
%env AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED=true

# Prepare workspace, experiment, and compute

In [ ]:
import azure.ml
from azure.ml import MLClient

SUBSCRIPTION_ID = '381b38e9-9840-4719-a5a0-61d9585e1e91'
RESOURCE_GROUP_NAME = 'lesam_centraluseuap_rg'
WORKSPACE_NAME = "lesam-master"
EXPERIMENT_NAME = "MySdkV2SweepJobExperiment"
CLUSTER_NAME = "cpu-cluster"

client = MLClient(SUBSCRIPTION_ID, RESOURCE_GROUP_NAME, workspace_name=WORKSPACE_NAME)

client

In [ ]:
# Make Compute
# choose a name for your cluster
from azure.ml.entities import AmlCompute

try:
    compute_target = client.compute.get(CLUSTER_NAME)
    print('Found existing compute target.')
except Exception:
    print('Creating a new compute target...')
    compute = AmlCompute(
        name=CLUSTER_NAME, size="STANDARD_NC6",
        min_instances=0, max_instances=4,
        idle_time_before_scale_down=120
    )

    # create the cluster
    # client.compute.create(compute)
    compute_target = client.compute.get(CLUSTER_NAME)

compute_target

# Prepare Sweep Job

In [ ]:
# Prepare Early Termination Policy
from azure.ml._restclient.v2021_03_01_preview.models import MedianStoppingPolicy
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

In [ ]:
# Prepare search space
from azure.ml.entities import Choice, Uniform, Randint, QUniform
search_space = {
    "y1": Choice([1111, 2222, 3333]),
    "y2": Uniform(0, 1),
    "y3": Uniform(0, 1)
}

In [ ]:
# Make SweepJob
from azure.ml.entities import SweepJob, ParameterizedCommand, Code, ComputeConfiguration, Environment
from azure.ml._restclient.v2021_03_01_preview.models import Objective

objective = Objective(goal="Maximize", primary_metric="Accuracy")
code = Code(local_path = ".", name = "foo", version = 19)
compute = ComputeConfiguration(target = compute_target.id)
# environment = client.environments.get(name='AzureML-TensorFlow-2.3-GPU', version=17)

In [ ]:
trial = ParameterizedCommand(
    command = 'python example_script.py --x1 {search_space.y1} --x2 {search_space.y2} --x3 {search_space.y3}',
    code = code,
    compute = compute,
    environment = 'AzureML-TensorFlow-2.3-GPU:17',
)

In [ ]:
sweep_job = SweepJob(
    algorithm = "Random",
    search_space = search_space,
    objective = objective,
    trial = trial,
    early_termination = early_termination_policy,
    max_concurrent_trials = 1,
    max_total_trials = 1,
    timeout_minutes = 50,
    experiment_name = EXPERIMENT_NAME
)

print(sweep_job)

In [ ]:
# Submit
created_job = client.jobs.create_or_update(sweep_job)
created_job
